<a href="https://colab.research.google.com/github/Oussama-Aouina/Deep-Convolutional-Generator-Adversarial-Network/blob/main/Deep_Convolutional_Generator_Adversarial_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function # print fun transform to 3.0 to 2.6+

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CV-TP-Projet

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CV-TP-Projet


In [ ]:
import torch
import torch.nn as nn # neural network
import torch.nn.parallel # run paralllel execution
import torch.optim as optim # optimizer
import torch.utils.data # import data in mini batches using Dataloader function
import torchvision.datasets as dst # datsets
import torchvision.transforms as transforms # create dependencies between 2 networks
import torchvision.utils as vutils # visualization
from torch.autograd import Variable # for working torch tensor we need torch variable

In [ ]:
batch_szie = 64
image_size = 64 # generated image size

In [ ]:
# We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.
transform = transforms.Compose([transforms.Resize(image_size) ,
                                transforms.ToTensor() ,
                                transforms.Normalize((0.5 , 0.5 , 0.5) , (0.5 , 0.5 , 0.5))
                               ])

In [ ]:
dataset = dst.CIFAR10(root = 'data' , download = True , transform = transform)

100%|██████████| 170M/170M [00:03<00:00, 42.9MB/s]


Extracting data/cifar-10-python.tar.gz to data


In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_szie , shuffle=True , num_workers=2)
# num worker means CPU multicore use for loading the dataset

In [ ]:
# initialize the weight init function which takes neural network as input
# and set all weights of Neural Network

def weights_init(m):
    classname = m.__class__.__name__ # getting class name

    if classname.find('Conv') != -1 : # if conv then set this weights
        m.weight.data.normal_(0.0 , 0.02)
    elif classname.find('Batchnorm') != -1: # if batchnoram then set this weights
        m.weight.data.normal_(1.0 , 0.02)
        m.bias.data.fill_(0) # bias fil with zero.

### Generator Architecture

In [ ]:
class G(nn.Module):

    def __init__(self):
        super(G , self).__init__() # calling super classs methos on this class future object 1st module

        # 2 nd meta module which consists of many layers or modules.
        self.main = nn.Sequential(

        # we are using inverse conv. layer for Generator because it genrates the images.

        # ConvTranspose2d(input , output , kernelsize , strides , padding , bias)
        nn.ConvTranspose2d(100 , 512 , 4 , 1 , 0 , bias = True),
        nn.BatchNorm2d(512), # we batchnormalize the output os 512 neurons
        nn.ReLU(True),

        nn.ConvTranspose2d(512 , 256 , 4 , 2 , 1 , bias = True),
        nn.BatchNorm2d(256),
        nn.ReLU(True),

        nn.ConvTranspose2d(256 , 128 , 4 , 2 , 1 , bias = True),
        nn.BatchNorm2d(128),
        nn.ReLU(True),

        nn.ConvTranspose2d(128 , 64 , 4 , 2 , 1 , bias = True),
        nn.BatchNorm2d(64),
        nn.ReLU(True),

        #last layer of Inverse conv. layer which genrate image so we set output = 3 which is color channels.
        nn.ConvTranspose2d(64 , 3 , 4 , 2 , 1 , bias = True),
        nn.Tanh() ) # tanh value between -1 to 1 and centerd near zero.

        # We have to generate image linearity between -1 and +1 and centerd zero
        # why is that ?
        # We have to make standard same as images of dataset becuase this G. images goes into D. network

    # forward propogation

    def forward(self , input):  # input is 100x100 noise vector which is useful for creating images
        output = self.main(input)
        return output

Object of Generator

In [ ]:
netG = G()
netG.apply(weights_init) # initilaize the weights

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1))
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (13): Tanh()
  )
)

### Discriminator Architecture

In [ ]:
class D(nn.Module):

    def __init__(self):
        super(D , self).__init__()

        self.main = nn.Sequential(

        # Conv2d(input , output , kernelsize , stride , padding , bias)
        nn.Conv2d(3 , 64 , 4 , 2 , 1, bias = True),

        # here we are not normalized first network and we also apply leakyrelu activaiotn fucntion
        nn.LeakyReLU(0.2 , inplace = True),

        nn.Conv2d(64 , 128 , 4 , 2 , 1 , bias = True),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(0.2 , True),

        nn.Conv2d(128 , 256 , 4 , 2 , 1 , bias = True),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(0.2 , True),

        nn.Conv2d(256 , 512 , 4 , 2 , 1 , bias = True),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(0.2 , True),

        # final layer is 1 neurons because discriminator discriminates the image wheather it is accepted ot not
        # 1 - Accepted
        # 0 - Rejected
        # finally we choose sigmoid becuase it genrates o/p between 0 and 1.

        nn.Conv2d(512 , 1 , 4 , 1 , 0 , bias = True),
        nn.Sigmoid()
        )

    def forward(self ,input):
        output = self.main(input)
        return output.view(-1) # d network is CNN so at last we flatten the layer so we simplt do that by this trick.

Object of Discriminator

In [ ]:
netD = D()
netD.apply(weights_init)

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1))
    (12): Sigmoid()
  )
)

### Training The Networks

In [ ]:
# creating loss
# In this case we are using binary cross loss

criterion = nn.BCELoss()
# Adam( parameters , learnning rate , coefficients)
optimizerG = optim.Adam(netG.parameters() , lr = 0.0002 , betas = (0.5 , 0.999))
optimizerD = optim.Adam(netD.parameters() , lr = 0.0002 , betas = (0.5 , 0.999))

In [ ]:
for epoch in range(10):
    # i index , data - mininbatches
    # collect data from dataloader and 0 staring range

    for i , data in enumerate(dataloader , 0):

        ### 1 St Step

        netD.zero_grad()

        ## Training on Real Images

        real , _ = data
        input = Variable(real) # Varibale special type which combinaitonof tensor and gradient
        target = Variable(torch.ones(input.size()[0])) # accepeted so ones array
        output = netD(input)
        errD_real = criterion(output , target)

        ## Training on Fake Images

        # creating noise images for every batch with size of 100. 100 features of 1x1
        noise = Variable(torch.randn(input.size()[0] , 100 , 1 , 1))
        fake = netG(noise) # input noise in G. network it output fake image that is out input
        target = Variable(torch.zeros(input.size()[0])) # rejected so zeros array
        output = netD(fake.detach()) # this means don't calculat gradient for this output save computational power for D. net only
        errD_fake = criterion(output , target)

        ## Backpropogating the error in D. network

        errD = errD_fake + errD_real # total loss
        errD.backward() # backpropogation calculate loss
        optimizerD.step()  # update the weights

        ### 2nd Step

        ## Train Generator

        netG.zero_grad()
        target = Variable(torch.ones(input.size()[0]))
        output = netD(fake)
        errG = criterion(output , target)

        ## backpropogation

        errG.backward()
        optimizerG.step()


        # printing some information

        print('[{} / {}][{} / {}] Loss_D: {} Loss_G: {}'.format(epoch, 10, i, len(dataloader), errD.data, errG.data))

        # saving the images after 100 steps.

        if i % 100 == 0:

            # vutils.save_image(real , '%s/real_sample.png' % 'results' , normalize = True)

            fake = netG(noise)

            vutils.save_image(fake.data , '%s/fake_sample_epoch_%03d.png' % ('results' , epoch) , normalize = True)


[0 / 10][0 / 782] Loss_D: 0.6187783479690552 Loss_G: 4.878893852233887
[0 / 10][1 / 782] Loss_D: 1.245884656906128 Loss_G: 10.953347206115723
[0 / 10][2 / 782] Loss_D: 0.2896219491958618 Loss_G: 10.123943328857422
[0 / 10][3 / 782] Loss_D: 0.13462014496326447 Loss_G: 6.416738986968994
[0 / 10][4 / 782] Loss_D: 0.9968058466911316 Loss_G: 12.591666221618652
[0 / 10][5 / 782] Loss_D: 0.2929350733757019 Loss_G: 12.445556640625
[0 / 10][6 / 782] Loss_D: 0.27377936244010925 Loss_G: 7.697700500488281
[0 / 10][7 / 782] Loss_D: 0.41373124718666077 Loss_G: 11.269068717956543
[0 / 10][8 / 782] Loss_D: 0.09177258610725403 Loss_G: 10.356197357177734
[0 / 10][9 / 782] Loss_D: 0.07121139019727707 Loss_G: 7.256308555603027
[0 / 10][10 / 782] Loss_D: 0.33330509066581726 Loss_G: 13.49902057647705
[0 / 10][11 / 782] Loss_D: 0.179002583026886 Loss_G: 13.550409317016602
[0 / 10][12 / 782] Loss_D: 0.18263432383537292 Loss_G: 9.534198760986328
[0 / 10][13 / 782] Loss_D: 0.04272764176130295 Loss_G: 5.69571542